In [1]:
!nvidia-smi

Thu Jan 21 08:57:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install simpletransformers
!pip install transformers -U

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (4.2.2)
     |████████████████████████████████| 2.9MB 14.3MB/s 
  Found existing installation: tokenizers 0.10.0
    Uninstalling tokenizers-0.10.0:
      Successfully uninstalled tokenizers-0.10.0


In [1]:
#restart instance before importing
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
import pandas as pd
import io
import pickle

In [2]:
#import of the files into colaboratory
from google.colab import files
uploaded = files.upload()

Saving test.json to test.json
Saving train.json to train.json
Saving train_label.csv to train_label.csv


### Import of the testing and training data set

In [3]:
train_df = pd.read_json('/content/train.json')
test_df = pd.read_json('/content/test.json')
submission_df = pd.read_csv('/content/train_label.csv')
train_labels = pd.read_csv('/content/train_label.csv')

In [4]:
train_df.drop(['Id','gender'], axis = 1, inplace = True )
train_df['label'] = train_labels['Category'].tolist()
train_df.columns = ['text','labels']
text = test_df['description'].tolist()

### Import of the BERT model into the notebook

In [5]:
model_args = ClassificationArgs(num_train_epochs=1)
model = ClassificationModel(
    'bert',
    'bert-base-cased',
    num_labels=28,
    args=model_args
) 

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

### Training the model

In [ ]:
model.train_model(train_df)

  0%|          | 0/217197 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/27150 [00:00<?, ?it/s]

### Predictions on the test set

In [ ]:
predictions, raw_outputs = model.predict(text)

  0%|          | 0/54300 [00:00<?, ?it/s]

  0%|          | 0/6788 [00:00<?, ?it/s]

In [ ]:
submission_df['Category'] = predictions
submission_df.to_csv('/content/bert_submission.csv', index = False)

In [ ]:
submission_df

,Id,Category
0,0,6
1,1,20
2,2,20
3,3,19
4,4,19
...,...,...
54295,54295,19
54296,54296,6
54297,54297,19
54298,54298,26
